# ---------------------------------------------------------
# 🚨 MASTER SETUP BLOCK - DO NOT EDIT THIS CELL
# ---------------------------------------------------------

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 1. Automatic Path Detection (Works for everyone)
# Assumes data is in a folder named 'data' in the root
file_path = os.path.join('data', 'US_Accidents.csv')

# 2. Load Full Data
print("Loading FULL dataset...")
df = pd.read_csv(file_path)

# 3. Standard Cleaning (Applied immediately)
# Convert time
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce')

# Extract standardized time features
df['Year'] = df['Start_Time'].dt.year
df['Month'] = df['Start_Time'].dt.month
df['Hour'] = df['Start_Time'].dt.hour
df['Weekday'] = df['Start_Time'].dt.day_name()

# Drop 'Noise' columns (Agreed upon by Squad Lead)
cols_to_drop = ['ID', 'Source', 'Zipcode', 'Country', 'Timezone', 'Airport_Code']
df.drop(columns=cols_to_drop, errors='ignore', inplace=True)

print("✅ Data Loaded & Pre-processed. Ready for Analysis!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
# ---------------------------------------------------------